In [1]:
# Spark Session
from pyspark.sql import SparkSession
import subprocess
import os
import platform

def get_hdfs_bin():
    hadoop_home = os.environ.get("HADOOP_HOME")
    if not hadoop_home:
        raise EnvironmentError("HADOOP_HOME environment variable is not set.")
    if platform.system() == "Windows":
        return os.path.join(hadoop_home, "bin", "hdfs.cmd")
    return os.path.join(hadoop_home, "bin", "hdfs")

# Supprimer le dossier si existant (forcer la suppression)
try:
    subprocess.run([get_hdfs_bin(), "dfs", "-rm", "-r", "-f", "/output/3/"], check=True)
    print("Dossier /output/3/ supprimé avec succès.")
except subprocess.CalledProcessError as e:
    print("Le dossier /output/3/ n'existe peut-être pas ou une erreur est survenue :", e)

# Initialiser SparkSession avec support Hadoop
spark = SparkSession.builder \
    .appName("Basic Transformation - II") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.hadoop.fs.hdfs.impl.disable.cache", "true") \
    .getOrCreate()

print("SparkSession initialisée avec succès.")

2026-01-04 19:17:36,786 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Dossier /output/3/ supprimé avec succès.


26/01/04 19:17:37 WARN Utils: Your hostname, OUARAS-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.78 instead (on interface en0)
26/01/04 19:17:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/04 19:17:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession initialisée avec succès.


In [2]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [3]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [4]:
# Show emp dataframe (ACTION)

emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [5]:
# Print Schema

emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [6]:
# Casting Column
# select employee_id, name, age, cast(salary as double) as salary from emp
from pyspark.sql.functions import col, cast

emp_casted = emp.select("employee_id", "name", "age", col("salary").cast("double"))

In [7]:
emp_casted.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)



In [8]:
# Adding Columns
# select employee_id, name, age, salary, (salary * 0.2) as tax from emp_casted

emp_taxed = emp_casted.withColumn("tax", col("salary") * 0.2)

In [9]:
emp_taxed.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [10]:
# Literals
# select employee_id, name, age, salary, tax, 1 as columnOne, 'two' as columnTwo from emp_taxed
from pyspark.sql.functions import lit

emp_new_cols = emp_taxed.withColumn("columnOne", lit(1)).withColumn("columnTwo", lit('two'))

In [11]:
emp_new_cols.show()

+-----------+-------------+---+-------+-------+---------+---------+
|employee_id|         name|age| salary|    tax|columnOne|columnTwo|
+-----------+-------------+---+-------+-------+---------+---------+
|        001|     John Doe| 30|50000.0|10000.0|        1|      two|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|      two|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|      two|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|      two|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|      two|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|      two|
|        007|James Johnson| 42|70000.0|14000.0|        1|      two|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|      two|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|      two|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|      two|
|        011|   David Park| 38|65000.0|13000.0|        1|      two|
|        012|   Susan Chen| 31|54000.0|10800.0| 

In [12]:
# Renaming Columns
# select employee_id as emp_id, name, age, salary, tax, columnOne, columnTwo from emp_new_cols

emp_1 = emp_new_cols.withColumnRenamed("employee_id", "emp_id")

In [13]:
emp_1.show()

+------+-------------+---+-------+-------+---------+---------+
|emp_id|         name|age| salary|    tax|columnOne|columnTwo|
+------+-------------+---+-------+-------+---------+---------+
|   001|     John Doe| 30|50000.0|10000.0|        1|      two|
|   002|   Jane Smith| 25|45000.0| 9000.0|        1|      two|
|   003|    Bob Brown| 35|55000.0|11000.0|        1|      two|
|   004|    Alice Lee| 28|48000.0| 9600.0|        1|      two|
|   005|    Jack Chan| 40|60000.0|12000.0|        1|      two|
|   006|    Jill Wong| 32|52000.0|10400.0|        1|      two|
|   007|James Johnson| 42|70000.0|14000.0|        1|      two|
|   008|     Kate Kim| 29|51000.0|10200.0|        1|      two|
|   009|      Tom Tan| 33|58000.0|11600.0|        1|      two|
|   010|     Lisa Lee| 27|47000.0| 9400.0|        1|      two|
|   011|   David Park| 38|65000.0|13000.0|        1|      two|
|   012|   Susan Chen| 31|54000.0|10800.0|        1|      two|
|   013|    Brian Kim| 45|75000.0|15000.0|        1|   

In [14]:
# Column names with Spaces
# select employee_id as emp_id, name, age, salary, tax, columnOne, columnTwo as `Column Two` from emp_new_cols

emp_2 = emp_new_cols.withColumnRenamed("columnTwo", "Column Two")

In [15]:
emp_2.show()

+-----------+-------------+---+-------+-------+---------+----------+
|employee_id|         name|age| salary|    tax|columnOne|Column Two|
+-----------+-------------+---+-------+-------+---------+----------+
|        001|     John Doe| 30|50000.0|10000.0|        1|       two|
|        002|   Jane Smith| 25|45000.0| 9000.0|        1|       two|
|        003|    Bob Brown| 35|55000.0|11000.0|        1|       two|
|        004|    Alice Lee| 28|48000.0| 9600.0|        1|       two|
|        005|    Jack Chan| 40|60000.0|12000.0|        1|       two|
|        006|    Jill Wong| 32|52000.0|10400.0|        1|       two|
|        007|James Johnson| 42|70000.0|14000.0|        1|       two|
|        008|     Kate Kim| 29|51000.0|10200.0|        1|       two|
|        009|      Tom Tan| 33|58000.0|11600.0|        1|       two|
|        010|     Lisa Lee| 27|47000.0| 9400.0|        1|       two|
|        011|   David Park| 38|65000.0|13000.0|        1|       two|
|        012|   Susan Chen| 31|540

In [16]:
# Remove Column

emp_dropped = emp_new_cols.drop("columnTwo", "columnOne")

In [17]:
emp_dropped.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        016|  Kelly Zhang| 30|49000.0| 9800.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        018|    Nancy Liu| 29|50000.0|1

In [18]:
# Filter data 
# select employee_id as emp_id, name, age, salary, tax, columnOne from emp_col_dropped where tax > 1000

emp_filtered = emp_dropped.where("tax > 10000")

In [19]:
emp_filtered.show()

+-----------+-------------+---+-------+-------+
|employee_id|         name|age| salary|    tax|
+-----------+-------------+---+-------+-------+
|        003|    Bob Brown| 35|55000.0|11000.0|
|        005|    Jack Chan| 40|60000.0|12000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|
|        007|James Johnson| 42|70000.0|14000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|
|        009|      Tom Tan| 33|58000.0|11600.0|
|        011|   David Park| 38|65000.0|13000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|
|        013|    Brian Kim| 45|75000.0|15000.0|
|        015|  Michael Lee| 37|63000.0|12600.0|
|        017|  George Wang| 34|57000.0|11400.0|
|        019|  Steven Chen| 36|62000.0|12400.0|
|        020|    Grace Kim| 32|53000.0|10600.0|
+-----------+-------------+---+-------+-------+



In [20]:
# LIMIT data
# select employee_id as emp_id, name, age, salary, tax, columnOne from emp_filtered limit 5

emp_limit = emp_filtered.limit(5)

In [21]:
# Show data

emp_limit.show(2)

+-----------+---------+---+-------+-------+
|employee_id|     name|age| salary|    tax|
+-----------+---------+---+-------+-------+
|        003|Bob Brown| 35|55000.0|11000.0|
|        005|Jack Chan| 40|60000.0|12000.0|
+-----------+---------+---+-------+-------+
only showing top 2 rows



In [22]:
# Bonus TIP
# Add multiple columns

columns = {
    "tax" : col("salary") * 0.2 ,
    "oneNumber" : lit(1), 
    "columnTwo" : lit("two")
}

emp_final = emp.withColumns(columns)

In [23]:
emp_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------+---------+---------+
|employee_id|department_id|         name|age|gender|salary| hire_date|    tax|oneNumber|columnTwo|
+-----------+-------------+-------------+---+------+------+----------+-------+---------+---------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|10000.0|        1|      two|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15| 9000.0|        1|      two|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|11000.0|        1|      two|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30| 9600.0|        1|      two|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|12000.0|        1|      two|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|10400.0|        1|      two|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|14000.0|        1|      two|
|        0

In [24]:
# Write data as CSV output (ACTION)

emp_final.write.format("csv").mode("overwrite").option("header", "true").save("hdfs://localhost:9000/output/3/emp.csv")

In [25]:
spark.stop()